train_val, which means training + validation, is used since we perform cross-validation

In [ ]:
%store -r X_train_val_scaled
%store -r Y_train_val
print(X_train_val_scaled.shape)
print(Y_train_val.shape)

(10896, 51)
(10896, 2)


In [ ]:
import polars as pl
from sklearn.model_selection import GridSearchCV, KFold
from sksurv.tree import SurvivalTree
import numpy as np
from sksurv.ensemble import RandomSurvivalForest

## Elastic Net

This is performed in the regression notebook.

## Survival Tree

In [ ]:
# define parameters for the cross validation
param_grid = {
    "splitter": ["best", "random"] ,
    "min_samples_split": [6, 3, 10],
    "max_depth": [2, 5, None],
}
cv = KFold(n_splits=5, shuffle=True, random_state=2054)
search = GridSearchCV(SurvivalTree(random_state=2054), param_grid, cv=cv, verbose=10) # grid search cross validation
search.fit(X_train_val_scaled, Y_train_val.to_numpy(structured=True)) # fit it in the validation and training set 

print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)

pl.DataFrame(search.cv_results_).select("param_splitter", "param_min_samples_split", "param_max_depth", "mean_test_score").sort("mean_test_score", descending=True)

## Random Survival Forest

In [ ]:
# define parameters for the random survival forest cross validation
param_grid = {
    "min_samples_split": [5, 3, 10] ,
    "n_estimators": [1000, 800, 500] ,
    "max_depth": [2, 5, None],
}
cv = KFold(n_splits=5, shuffle=True, random_state=2054)
search = GridSearchCV(RandomSurvivalForest(random_state=2054,  n_jobs = -1), param_grid, cv=cv)
search.fit(X_train_val_scaled, Y_train_val.to_numpy(structured=True)) # fit it in the validation and train

print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)

pl.DataFrame(search.cv_results_).select("param_min_samples_split", "param_n_estimators", "param_max_depth", "mean_test_score").sort("mean_test_score", descending=True)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=cda3dc7f-2383-4ca3-97fb-1b86671fd3dc' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>